<a href="https://colab.research.google.com/github/IgorCzudy/whatsapp_message_clasterization/blob/main/notebooks/GC_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers datasets evaluate
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.9 MB/s eta 0:00:00


# Prepare data

In [3]:
import pandas as pd
music_aug = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/finalData/music_augmented.csv")
party_aug = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/finalData/party_augmented.csv")
full = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/CleanChats/FullChat.csv")

In [4]:
full.rename(columns={"Message": "message", "Section": "section"}, inplace=True)

In [5]:
full = full.drop(columns=["Date", "Time", "Sender"])

sections = ["Football", "Music", "Party","General"]
final_df = full[full["section"].isin(sections)]

music_aug["section"] = "Music"
party_aug["section"] = "Party"

final_df = pd.concat([final_df, music_aug, party_aug])

In [6]:
# remove duplicates
final_df = final_df.drop_duplicates()
final_df = final_df.reset_index(drop=True)

In [7]:
final_df["section"].value_counts()

Football    3415
Party       2715
General     2636
Music       1480
Name: section, dtype: int64

In [8]:
final_df=final_df.rename(columns={'section': 'label','message':'text'})
final_df['label'] = final_df['label'].replace('General', 0)
final_df['label'] = final_df['label'].replace('Football', 1)
final_df['label'] = final_df['label'].replace('Party', 2)
final_df['label'] = final_df['label'].replace('Music', 3)
final_df.head()

,text,label
0,Someone going to the beach this afternoon?,0
1,We'll be a few minutes late \nSiuuuuuuu,0
2,locatie: https://maps.google.com/?q=39.473291...,0
3,"Hii, I also will be a few minutes late for th...",0
4,Me too,0


## Tokenize

In [9]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import Dataset

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

data = Dataset.from_pandas(final_df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_dataset = data.map(preprocess_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10246 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset[4000]


{'text': ' Because of the tenis player?😂',
 'label': 1,
 'input_ids': [101, 2138, 1997, 1996, 2702, 2483, 2447, 1029, 100, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:

# column we want to stratify with respect to
stratify_column_name = "label"
# create class label column and stratify
splits = tokenized_dataset.class_encode_column(stratify_column_name).train_test_split(test_size=0.2,stratify_by_column=stratify_column_name)
dataset_train = splits["train"]
dataset_test = splits["test"]
print(f"Train : {dataset_train}")
print(f"Test : {dataset_test}")

Stringifying the column:   0%|          | 0/10246 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10246 [00:00<?, ? examples/s]

Train : Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8196
})
Test : Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2050
})


# Model

In [12]:
# track accuracy during training
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
f1m = evaluate.load("f1")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    rc = recall.compute(predictions=predictions, references=labels,average='macro')["recall"]
    f1 = f1m.compute(predictions=predictions, references=labels,average='macro')["f1"]
    pcs = precision.compute(predictions=predictions, references=labels,average='macro')["precision"]
    return {"accuracy":acc,"precision":pcs,"recall":rc,"f1":f1}

# Login to publish model to hugginface


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

id2label = {0: "GENERAL", 1: "FOOTBALL", 2: "PARTY", 3: "MUSIC"}
label2id = {"GENERAL": 0, "FOOTBALL": 1, "PARTY": 2, "MUSIC": 3}

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="whatsapp-group-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.582500,0.411592,0.842439,0.868877,0.845608,0.854767
2,0.339100,0.389757,0.855122,0.875712,0.865654,0.869430
3,0.247600,0.491532,0.853171,0.865927,0.862838,0.864077
4,0.163700,0.553140,0.855610,0.873291,0.867114,0.869371
5,0.117300,0.642293,0.860488,0.875876,0.869110,0.872286


TrainOutput(global_step=2565, training_loss=0.2857907828764144, metrics={'train_runtime': 595.2693, 'train_samples_per_second': 68.843, 'train_steps_per_second': 4.309, 'total_flos': 1240605292296864.0, 'train_loss': 0.2857907828764144, 'epoch': 5.0})

To publish to hugginface need login with the access token

In [16]:
trainer.push_to_hub()

events.out.tfevents.1701014891.561e80f6b13a.208.0:   0%|          | 0.00/7.92k [00:00<?, ?B/s]

'https://huggingface.co/DTempo/whatsapp-group-classifier/tree/main/'

## Inference

In [ ]:
from transformers import pipeline
import transformers
#Change model to the one you want to use
model = "Dtempo/whatsapp-group-classifier"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline("text-classification",model=model)

sequences = pipeline(["Hey everyone! There's a jazz concert at Jimmy Glass Jazz Bar this Friday night. Who's up for some  tunes?",
                      "Lest go do some running tomorrow ",
                      "Hello, any plans for tonight? I heard Bamboo Pub has a free entry until 00:30. What do you think?"])
for seq in sequences:
    print(f"Result: {seq}")


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Result: {'label': 'MUSIC', 'score': 0.9926152229309082}
Result: {'label': 'FOOTBALL', 'score': 0.9275616407394409}
Result: {'label': 'PARTY', 'score': 0.9916572570800781}
